In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import distance,lonlat
import math
import geopandas as gpd
from shapely.geometry import Point

In [74]:
fire_data=pd.read_csv("../data/output_data/output_fire_data/clustering_fire_data.csv")
fire_data

,latitude,longitude,frp
0,-36.979083,143.869362,9.0278
1,-37.395876,148.702175,61.1422
2,-36.360385,141.245716,10.0394
3,-37.326302,146.998582,17.1700
4,-37.521991,142.326735,13.3154
...,...,...,...
95,-35.033808,143.126835,1.2592
96,-38.101039,141.752253,6.5914
97,-34.821044,142.132761,3.4122
98,-38.242735,146.371048,21.8620


In [124]:
def obj_func(drone_cors):
    res=0
    for i in range(len(fire_data)):
        fire_point_monitor=0.0
        for j in range(len(drone_cors)):
            dis=distance(lonlat(drone_cors[j,0],drone_cors[j,1]),lonlat(float(fire_data.loc[i,['longitude']]),float(fire_data.loc[i,['latitude']]))).km
            if(dis<30):
                fire_point_monitor+=1/dis
        if fire_point_monitor==0:
            res+=float(fire_data.loc[i,['frp']])
        else:
            res+=(1/fire_point_monitor)*float(fire_data.loc[i,['frp']])
    return float(res)

In [76]:
def random_blur(x_cor,y_cor):
    angle=np.random.rand()*math.pi*2
    return (x_cor+math.cos(angle),y_cor+math.sin(angle))

In [77]:
a=drone_cors.copy()
a[0,:]=(0.1,0.2)
type(a[0,:])

numpy.ndarray

In [78]:
len(drone_cors)

80

In [79]:
drone_number=80
rec_max_y=-33.65
rec_min_y=-39.57
rec_max_x=150.15
rec_min_x=140.67

drone_cors=np.random.random((drone_number,2))

In [80]:
file_path='../data/output_data/v_boundary.shp'
Victoria=gpd.read_file(file_path)

In [81]:
count=0
while True:
    if count==drone_number:
        break
    lon=np.random.rand()*(rec_max_y-rec_min_y)+rec_min_y
    lat=np.random.rand()*(rec_max_x-rec_min_x)+rec_min_x
    point=Point(lat,lon)
    if Victoria.geometry.contains(point).bool():
        drone_cors[count,0]=lat
        drone_cors[count,1]=lon
        count+=1

drone_cors

array([[143.01420035, -34.74745101],
       [145.5820293 , -38.5968623 ],
       [147.31638416, -37.82509127],
       [144.66773613, -37.93694039],
       [142.6376798 , -37.05722822],
       [144.23457666, -36.75469975],
       [144.92007815, -36.56436663],
       [143.20908734, -35.87900197],
       [141.7056892 , -37.08441593],
       [143.91282803, -37.0429738 ],
       [144.91627598, -37.63577805],
       [142.41892875, -36.4543462 ],
       [146.47446734, -35.98542708],
       [141.90626719, -36.01257058],
       [143.63053938, -37.77313089],
       [146.06793106, -38.45843827],
       [141.80833523, -38.0658996 ],
       [146.22675915, -36.87207153],
       [144.20888148, -37.69328778],
       [142.58224795, -35.22566552],
       [147.74465208, -36.17100179],
       [145.03817137, -38.42643378],
       [145.14474058, -37.09909854],
       [145.78073975, -37.80802832],
       [141.37508063, -37.35311693],
       [143.48050783, -36.92258316],
       [142.91822649, -35.59905636],
 

In [127]:
start_temp=200
decay_constant=1.0001
end_temp=10
temp=start_temp
temp_old=start_temp
obj_value=10000000.0
best_result=[]
while temp>end_temp:
    temp=temp*decay_constant
    drone_cors_old=drone_cors.copy()
    obj_value_old=obj_value

    for i in range(len(drone_cors)):
        drone_cors[i,:]=random_blur(drone_cors[i,0],drone_cors[i,1])
    obj_value=obj_func(drone_cors)
    print(obj_value)
    if(obj_value<obj_value_old):
        best_result.append(obj_value)
    else:
        obj_value=obj_value_old
        drone_cors=drone_cors_old.copy()
    

8987.760020256042
7141.711061368027
7148.012249602394
8316.80832988039
7871.20695734346
11382.855208518953
5103.8359913104
5412.642373806272
9108.51981749388
10454.71167160831
8797.610768451645
14895.950663411482
5499.858317009385
12090.141041804502
11544.335508716696
9202.517023275757
5383.825864165428
3907.8791949378115
7276.62363815009
9201.151877998744
12497.669287009147
6975.947427757373


KeyboardInterrupt: 